隐马尔可夫模型(HMM-Hidden Markov Models)
===

# 1.问题的引入
- 确定性的生成模式: 状态的转变是固定的，如红绿灯的变化规律，下一状态是已知的
- 非确定的生成模式: 状态的改变却是不确定的，如根据今天的天气推测明天的天气，这个状态的变化是不明确的

但是我们希望研究非确定性的生成模式，一种办法就是假设这个模型的每个状态都只依赖于前一个的状态，这个假设被称为马尔科夫假设

# 2.马尔可夫模型
想要了解隐马尔可夫模型，就需要先了解马尔可夫模型

## 2.1.马尔可夫过程
马尔可夫过程代表数学中具有马尔可夫性质的离散随机过程。在该过程中，每个状态的转移只依赖于之前的n个状态，这个过程被称为1个n阶的模型，其中n是影响转移状态的数目。最简单的马尔科夫过程就是一阶过程，每一个状态的转移只依赖于其之前的那一个状态。注意这和确定性系统不一样，因为这种转移是有概率的，而不是确定性的

## 2.2.马尔可夫链
马尔可夫链是随机变量$X_1,X_2,...,X_n$的一个数列。这些变量的范围，即他们所有可能取值的集合，被称为“状态空间”，而$X_n$的值则是在时间n的状态。如果$X_{n+1}$对于过去状态的条件概率分布仅是$X_n$的一个函数，则$P(X_{n+1}=x|X_0,X_1,...,X_n)=P(X_{n+1}=x|X_n)$。这里x为过程中的某个状态，这个恒等式可以看作是马尔可夫性质.在马尔可夫模型中，每一个状态都是可观察的序列，是状态关于时间的随机过程，也称为可视马尔可夫模型

## 2.3.状态转移概率与状态转移矩阵
一个含有N个状态的一阶过程有$N^2$个状态转移。每一个转移的概率叫做状态转移概率,就是从一个状态转移到另一个状态的概率.这所有的$N^2$个概率可以用一个状态转移矩阵来表示，其表示形式如下
$$A=\begin{bmatrix}
\alpha_{11}&\alpha_{12}&...&\alpha_{1N}\\
\alpha_{21}&\alpha_{22}&...&\alpha_{2N}\\
\vdots&\vdots&\vdots&\vdots\\
\alpha_{N1}&\alpha_{N2}&...&\alpha_{NN}
\end{bmatrix}$$
其中的约束条件为:$\begin{cases}
\alpha_{ij}=P(q_t=j|q_{t-1}=i), i \geq 1,j \leq N\\
\alpha_{ij} \geq 0\\
\sum_{j=1}^N\alpha_{ij}=1
\end{cases}$

## 2.4.举例说明
对于上面的天气的例子，首先定义三个状态：$S_1$下雨,$S_2$多云,$S_3$晴。然后定义初始向量$\overline{\nu}=(0.0,0.0,1.0)$，分别表示下雨的概率，阴天的概率以及晴天的概率。最后定义状态转移矩阵:$\begin{bmatrix}
0.4&0.3&0.3\\
0.2&0.6&0.2\\
0.1&0.1&0.8
\end{bmatrix}$,这个矩阵表示，如果昨天下雨，那么今天有40%的可能是下雨，30%的概率是阴天，30%的概率是晴天，很明显，矩阵中每一行的和都是1。那么根据这一模型,在今后七天中天气为"晴晴雨雨晴云晴"的概率为：
$$\begin{eqnarray}
\overline{\nu}&=&(0.0,0.0,1.0)\\
&\therefore& P(O|M)=P(S_3,S_3,S_3,S_1,S_1,S_3,S_2,S_3|M)\\
&=&P(S_3)*P(S_3|S_3)*P(S_3|S_3)*P(S_1|S_3)*P(S_1|S_1)*P(S_3|S_1)*P(S_2|S_3)*P(S_3|S_2)\\
&=&1 * \alpha_{33} * \alpha_{33} * \alpha_{31} * \alpha_{11} * \alpha_{13} * \alpha_{32} * \alpha_{23}\\
&=&0.8 * 0.8 * 0.1 * 0.4 * 0.3 * 0.1 * 0.2=1.536 * 10^{-4}
\end{eqnarray}$$
具体来解释一下:第一个$S_3$表示初始化向量，表示今天是晴天，而题目要求的是"晴晴雨雨晴云晴"的概率，也就是要求今天是晴明天也是晴的概率，明天是晴后天也是晴的概率，后天是晴，大后天是雨的概率,....，这些概率的联合概率

# 3.隐马尔可夫模型
在某些情况下，马尔科夫过程不足以描述我们希望发现的模式.例如，一个隐居的人可能不能直观的观察到天气的情况，但是民间传说告诉我们海藻的状态在某种概率上是和天气的情况相关的。在这种情况下我们有两个状态集合，一个可以观察到的状态集合（海藻的状态）和一个隐藏的状态（天气状况）。我们希望能找到一个算法可以根据海藻的状况和马尔科夫假设来预测天气的状况。这个时候就引入了隐马尔可夫模型。隐马尔可夫模型 (Hidden Markov Model) 是一种统计模型，用来描述一个含有隐含未知参数的马尔可夫过程。其难点是从可观察的参数中确定该过程的隐含参数，然后利用这些参数来作进一步的分析。下图是一个三个状态的隐马尔可夫模型状态转移图，其中x 表示隐含状态，y 表示可观察的输出，a 表示状态转换概率，b 表示输出概率<br/>
![images](images/01_12_001.jpg)<br/>
下图显示了天气的例子中隐藏的状态和可以观察到的状态之间的关系。我们假设隐藏的状态是一个简单的一阶马尔科夫过程，并且他们两两之间都可以相互转换<br/>
![images](images/01_12_002.gif)<br/>
在MM(Markov Model)中，每一个状态代表一个可观察的事件；在HMM(Hidden Markov Model)中观察到的时间是状态的随机函数，因此该模型是一双重随机过程，其中状态转移过程是不可观察的，而可观察的事件的随机过程是隐蔽的状态转换过程的随机函数。隐藏的状态和可以观察到的状态之间有一种概率上的关系，也就是说某种隐藏状态H被认为是某个可以观察的状态$O_1$是有概率的，假设为$P(O_1|H)$。如果可以观察的状态有三种，那么很显然$P(O_1|H)+P(O_2|H)+P(O_3|H)=1$。这样，我们也可以得到一个另一个矩阵，称为混淆矩阵。这个矩阵的内容是某个隐藏的状态被分别观察成集中不同的可以观察的状态的概率，在天气的例子中，这个矩阵如下图：<br/>
![images](images/01_12_003.png)

## 3.1.隐马尔可夫模型的三个假设
对于一个随机事件，有一个观察值序列：$O=O_1,O_2,...,O_T$，该事件因含着一个状态序列：$Q=q_1,q_2,...,q_T$。
- 假设1：马尔可夫性假设(状态构成一阶马尔可夫链):$P(q_i|q_{i-1}...q_1)=P(q_i|q_{i-1})$
- 假设2：不动性假设，即状态与具体时间无关.$P(q_{i+1}|q_i)=P(q_{j+1}|q_j),\forall{i,j}$
- 假设3：输出独立性假设，即输出仅与当前状态有关.$q(O_1,...,O_T|q_1,...,q_T)=\prod_p(O_t|q_t)$

## 3.2.隐马尔可夫模型的定义
一个HMM是有一个五元组描述的:$\lambda=(N,M,A,B,\pi)$,其中：
- $N=\{q_1,...,q_N\}$：状态的有限集合,也就是系统的(真实)状态，可以由一个马尔可夫过程进行描述，比如天气
- $M=\{v_1,...,v_M\}$:观察值的有限集合，也就是这个过程中"可视"的状态，比如海藻的湿度
- $A=\{\alpha_{ij}\}, \alpha_{ij}=P(q_t=S_j|q_{t-1}=S_i)$:状态转移概率矩阵，包含了一个隐藏状态到另一个隐藏状态的概率
- $B=\{b_{jk}\},b_{jk}=P(O_t=v_k|q_t=S_j)$：混淆矩阵，包含了给定隐马尔可夫模型的某一个特殊的隐藏状态，观察到的某个观察状态的概率
- $\pi=\{\pi_i\}, \pi_i=P(q_1=S_i)$：初始状态概率分布，包含了(隐)模型在时间t=1时一个特殊的隐藏状态的概率，也就是初始概率

## 3.3.观察序列产生步骤
给定HMM模型$\lambda=(A,B,\pi)$,则观察序列$O=O_1,O_2,...,O_T$可由以下步骤产生：
1. 根据初始状态概率的你$\pi=\pi_i$，选择一个初始状态$q_1=S_i$；
2. 设t=1
3. 根据状态$S_i$的输出概率分布$b_{jk}$，输出$O_t=v_k$
4. 根据状态转移概率分布$\alpha_{ij}$，转移到新状态$q_{t+1}=S_j$
5. 设t=t+1,如果t < T,重复步骤3、4，否则结束

## 3.4.HMM的三个基本问题
令$\lambda=\{\pi,A,B\}$为给定HMM的参数，令$O=O_1,O_2,...,O_T$为观察值序列，则有关于HMM的三个基本问题：
1. 评估问题：对于给定模型，求某个观察值序列的概率$P(O|\lambda)$，使用前向算法求解
2. 解码问题：对于给定模型和观察值序列，求可能性最大的状态序列$max_Q\{P(Q|O, \lambda)\}$，使用Viterbi算法求解
3. 学习问题：对于给定的一个观察值序列O，调整参数$\lambda$，使得观察值出现的概率$P(O|\lambda)$最大，使用前向-后向算法求解

# 4.HMM解法举例
题目一：抛硬币<br/>
三枚硬币，随机选择一枚，进行抛掷并记录抛掷的结果。可以描述未一个三个状态的HMM模型$\lambda$。$\lambda=(A,B,\pi)$，其中$S=\{1,2,3\}$,$V=\{H,T\}$，A如下表表示<br/>
![images](images/01_12_004.png)<br/>
B如下表表示<br/>
![images](images/01_12_005.png)<br/>
$\pi=\{\frac{1}{3},\frac{1}{3},\frac{1}{3}\}$
- 问题一：给定上述模型，观察到$O=(HHT)$的概率是多少？
- 问题二：给定上述模型，若观察到上述抛掷结果，最可能的硬币选择序列(状态转换序列)是什么？
- 问题三：若上述模型中的状态转移矩阵A、状态输出概率B和初始状态分布$\pi$均未知，如何根据观察序列得到它们？

题目二：<br/>
假设有状态集$N={S_1,S_2,S_3}$，观察序列集$M=\{A,B\}$，状态转移图如下：<br/>
![images](images/01_12_006.png)<br/>
$\pi=\{1,0,0\}$
- 问题一：给定上述模型，观察到$O=\{ABAB\}$的概率是多少？
- 问题二：给定上述模型以及观察序列O，问与O相对应的最佳状态序列Q是什么？

## 4.1.评估问题 - 前向算法
假设一个长度为T的观察序列为$(Y_{k_1},Y_{k_2},......,Y_{k_T})$,每一个y就是一个观察状态。在t=1时刻的中间结点的部分状态可以用下面的公式计算：$\alpha_1(j)=\pi(j)*b_{jk_1}$，对于t>1的情况，公式为:$\alpha_{t+1}(j)=\sum_{i=1}^n(\alpha_t(i)\alpha_{ij})b_{jk_{t+1}}$.这样我们就可以用递归的方式来计算所有可能的路径的概率和，最后，所有的部分概率的计算公式为:$P_r(Y^{(k)})=\sum_{j=1}^n\alpha_T(j)$

### 4.1.1.题目一的问题一求解
t=1，状态为H:
$$\begin{eqnarray}
\alpha_1(1)&=&\pi_1 * b_1{H}=\frac{1}{3} * 0.5=0.1667\\
\alpha_1(2)&=&\pi_2 * b_1{H}=\frac{1}{3} * 0.75=0.25\\
\alpha_1(3)&=&\pi_3 * b_1{H}=\frac{1}{3} * 0.25=0.0833
\end{eqnarray}$$
t=2,状态为H：
$$\begin{eqnarray}
\alpha_2(1)&=&(\alpha_1(1)\alpha_{11}+\alpha_1(2)\alpha_{21}+\alpha_1(3)\alpha_{31})*b_1{H}=(0.1667*0.9+0.25*0.45+0.0833*0.45)*0.5=(0.15+0.1125+0.0375)*0.5=0.1500\\
\alpha_2(2)&=&(\alpha_1(1)\alpha_{12}+\alpha_1(2)\alpha_{22}+\alpha_1(3)\alpha_{32})*b_2{H}=(0.1667*0.05+0.25*0.1+0.0833*0.45)*0.75=(0.0083+0.025+0.0375)*0.75=0.0531\\
\alpha_2(3)&=&(\alpha_1(1)\alpha_{13}+\alpha_1(2)\alpha_{23}+\alpha_1(3)\alpha_{33})*b_3{H}=(0.1667*0.05+0.25*0.45+0.0833*0.1)*0.25=(0.0083+0.1125+0.0083)*0.25=0.0323
\end{eqnarray}$$
t=3,状态为T：
$$\begin{eqnarray}
\alpha_3(1)&=&(\alpha_2(1)\alpha_{11}+\alpha_2(2)\alpha_{21}+\alpha_2(3)\alpha_{31})*b_1{T}=(0.1500*0.9+0.0531*0.45+0.0323*0.45)*0.5=(0.135+0.0239+0.0145)*0.5=0.0867\\
\alpha_3(2)&=&(\alpha_2(1)\alpha_{12}+\alpha_2(2)\alpha_{22}+\alpha_2(3)\alpha_{32})*b_2{T}=(0.1500*0.05+0.0531*0.1+0.0323*0.45)*0.25=(0.0075+0.0053+0.0145)*0.25=0.0068\\
\alpha_3(3)&=&(\alpha_2(1)\alpha_{13}+\alpha_2(2)\alpha_{23}+\alpha_2(3)\alpha_{33})*b_3{T}=(0.1500*0.05+0.0531*0.45+0.0323*0.1)*0.75=(0.0075+0.0239+0.0032)*0.75=0.0260\\
&\therefore& P(HHT|\lambda)=\alpha_3(1)+\alpha_3(2)+\alpha_3(3)=0.0867+0.0068+0.0260=0.1195
\end{eqnarray}$$

### 4.1.2.题目二问题一求解
t=1，状态为A:
$$\begin{eqnarray}
\alpha_1(1)&=&\pi_1 * b_1{A}=1 * 0.7=0.7\\
\alpha_1(2)&=&\pi_2 * b_2{A}=0 * 0.4=0\\
\alpha_1(3)&=&\pi_3 * b_3{A}=0 * 0.8=0
\end{eqnarray}$$
t=2,状态为B：
$$\begin{eqnarray}
\alpha_2(1)&=&(\alpha_1(1)\alpha_{11}+\alpha_1(2)\alpha_{21}+\alpha_1(3)\alpha_{31})*b_1{B}=(0.7*0.4+0*0+0*0)*0.3=0.28*0.3=0.084\\
\alpha_2(2)&=&(\alpha_1(1)\alpha_{12}+\alpha_1(2)\alpha_{22}+\alpha_1(3)\alpha_{32})*b_2{B}=(0.7*0.6+0*0.8+0*0)*0.6=0.42*0.6=0.252\\
\alpha_2(3)&=&(\alpha_1(1)\alpha_{13}+\alpha_1(2)\alpha_{23}+\alpha_1(3)\alpha_{33})*b_3{B}=(0.7*0+0*0.2+0*1)*0.2=0
\end{eqnarray}$$
t=3,状态为A：
$$\begin{eqnarray}
\alpha_3(1)&=&(\alpha_2(1)\alpha_{11}+\alpha_2(2)\alpha_{21}+\alpha_2(3)\alpha_{31})*b_1{A}=(0.084*0.4+0.252*0+0*0)*0.7=0.084*0.4*0.7=0.0235\\
\alpha_3(2)&=&(\alpha_2(1)\alpha_{12}+\alpha_2(2)\alpha_{22}+\alpha_2(3)\alpha_{32})*b_2{A}=(0.084*0.6+0.252*0.8+0*0)*0.4=(0.0504+0.2016)*0.4=0.0718\\
\alpha_3(3)&=&(\alpha_2(1)\alpha_{13}+\alpha_2(2)\alpha_{23}+\alpha_2(3)\alpha_{33})*b_3{A}=(0.084*0+0.252*0.2+0*1)*0.8=0.252*0.2*0.8=0.0403
\end{eqnarray}$$
t=4,状态为B：
$$\begin{eqnarray}
\alpha_4(1)&=&(\alpha_3(1)\alpha_{11}+\alpha_3(2)\alpha_{21}+\alpha_3(3)\alpha_{31})*b_1{B}=(0.0235*0.4+0.1008*0+0.0403*0)*0.3=0.0235*0.4*0.3=0.0028\\
\alpha_4(2)&=&(\alpha_3(1)\alpha_{12}+\alpha_3(2)\alpha_{22}+\alpha_3(3)\alpha_{32})*b_2{B}=(0.0235*0.6+0.1008*0.8+0.0403*0)*0.4=(0.0141+0.0806)*0.6=0.0568\\
\alpha_4(3)&=&(\alpha_3(1)\alpha_{13}+\alpha_3(2)\alpha_{23}+\alpha_3(3)\alpha_{33})*b_3{B}=(0.0235*0+0.1008*0.2+0.0403*1)*0.2=(0.02016+0.0403)*0.2=0.0121\\
&\therefore& P(ABAB|\lambda)=\alpha_4(1)+\alpha_4(2)+\alpha_4(3)=0.0028+0.0568+0.0121=0.1186
\end{eqnarray}$$

## 4.2.解码问题-Viterbi算法
解码问题即是确定最优路径问题。定义$\delta(t,i)$为t时间步沿状态序列$q_1,...,q_t$且$q_t=S_i$产生出$o_1,...o_t$的最大概率，即：
$$\delta(t,i)=max_{q_1,...q_{t-1}}P(q_1,...,q_{t-1},q_t=S_i,o_1,...,o_t|\lambda)$$
解法：
- 初始化：$\delta_1(i)=\pi_ib_i(O_1), \varphi_1=0, 1 \leq i \leq N$
- 递归：$\begin{cases}
\delta_t(j)=[max_{1 \leq i \leq N}\delta_{t-1}(i)\alpha_{ij}]b_j(O_t) & 2 \leq t \leq T, 1 \leq j \leq N\\
\varphi_t(j)=argmax_{1 \leq i \leq N}[\delta_{t-1}(i)\alpha_{ij}]b_j(O_t) & 2 \leq t \leq T, 1 \leq j \leq N
\end{cases}$
- 终结：$\begin{cases}
p^*=max_{1 \leq i \leq N}[\delta_T(i)]\\
q_T^*=argmax_{1 \leq i \leq N}[\delta_T(i)]
\end{cases}$
- 路径回溯：$q_t^*=\varphi_{t+1}(q_{t+1}^*), t=T-1,T-2,...,1$

### 4.2.1 题目一问题二求解
t=1，状态为H:
$$\begin{eqnarray}
\alpha_1(1)&=&\pi_1 * b_1{H}=\frac{1}{3} * 0.5=0.1667\\
\alpha_1(2)&=&\pi_2 * b_1{H}=\frac{1}{3} * 0.75=0.25\\
\alpha_1(3)&=&\pi_3 * b_1{H}=\frac{1}{3} * 0.25=0.0833
\end{eqnarray}$$
t=1时，不存在任何部分最优解<br/>
t=2,状态为H：
$$\begin{eqnarray}
\alpha_2(1)&=&max(\alpha_1(1)\alpha_{11},\alpha_1(2)\alpha_{21},\alpha_1(3)\alpha_{31})*b_1{H}=max(0.1667*0.9,0.25*0.45,0.0833*0.45)*0.5=max(0.15,0.1125,0.0375)*0.5=0.15*0.5=0.075\\
\alpha_2(2)&=&max(\alpha_1(1)\alpha_{12},\alpha_1(2)\alpha_{22},\alpha_1(3)\alpha_{32})*b_2{H}=max(0.1667*0.05,0.25*0.1,0.0833*0.45)*0.75=max(0.0083,0.025,0.0375)*0.75=0.0375*0.75=0.0281\\
\alpha_2(3)&=&max(\alpha_1(1)\alpha_{13},\alpha_1(2)\alpha_{23},\alpha_1(3)\alpha_{33})*b_3{H}=max(0.1667*0.05,0.25*0.45,0.0833*0.1)*0.25=max(0.0083,0.1125,0.0083)*0.25=0.0125*0.25=0.0031
\end{eqnarray}$$
t=3,状态为T：
$$\begin{eqnarray}
\alpha_3(1)&=&max(\alpha_2(1)\alpha_{11},\alpha_2(2)\alpha_{21},\alpha_2(3)\alpha_{31})*b_1{T}=max(0.0075*0.9,0.0281*0.45,0.0031*0.45)*0.5=max(0.0068,0.0126,0.0014)*0.5=0.0126*0.5=0.0063\\
\alpha_3(2)&=&max(\alpha_2(1)\alpha_{12},\alpha_2(2)\alpha_{22},\alpha_2(3)\alpha_{32})*b_2{T}=max(0.0075*0.05,0.0281*0.1,0.0031*0.45)*0.25=max(0.0004,0.0029,0.0014)*0.25=0.0029*0.25=0.0007\\
\alpha_3(3)&=&max(\alpha_2(1)\alpha_{13},\alpha_2(2)\alpha_{23},\alpha_2(3)\alpha_{33})*b_3{T}=max(0.0075*0.05,0.0281*0.45,0.0031*0.1)*0.75=max(0.0004,0.0126,0.0003)*0.75=0.0126*0.75=0.0095
\end{eqnarray}$$
$\therefore 1-3-3$最优

### 4.2.2.题目二问题二求解
t=1，状态为A:
$$\begin{eqnarray}
\alpha_1(1)&=&\pi_1 * b_1{A}=1 * 0.7=0.7\\
\alpha_1(2)&=&\pi_2 * b_2{A}=0 * 0.4=0\\
\alpha_1(3)&=&\pi_3 * b_3{A}=0 * 0.8=0\\
\psi_1(1)&=&0
\end{eqnarray}$$
t=2,状态为B：
$$\begin{eqnarray}
\alpha_2(1)&=&max(\alpha_1(1)\alpha_{11},\alpha_1(2)\alpha_{21},\alpha_1(3)\alpha_{31})*b_1{B}=max(0.7*0.4,0*0,0*0)*0.3=0.28*0.3=0.084, \psi_2(1)=1\\
\alpha_2(2)&=&max(\alpha_1(1)\alpha_{12},\alpha_1(2)\alpha_{22},\alpha_1(3)\alpha_{32})*b_2{B}=max(0.7*0.6,0*0.8,0*0)*0.6=0.42*0.6=0.252, \psi_2(2)=1\\
\alpha_2(3)&=&max(\alpha_1(1)\alpha_{13},\alpha_1(2)\alpha_{23},\alpha_1(3)\alpha_{33})*b_3{B}=max(0.7*0+0*0.2+0*1)*0.2=0
\end{eqnarray}$$
t=3,状态为A：
$$\begin{eqnarray}
\alpha_3(1)&=&max(\alpha_2(1)\alpha_{11},\alpha_2(2)\alpha_{21},\alpha_2(3)\alpha_{31})*b_1{A}=max(0.084*0.4,0.252*0,0*0)*0.7=0.084*0.4*0.7=0.0235, \psi_3(1)=1\\
\alpha_3(2)&=&max(\alpha_2(1)\alpha_{12},\alpha_2(2)\alpha_{22},\alpha_2(3)\alpha_{32})*b_2{A}=max(0.084*0.6,0.252*0.8,0*0)*0.4=max(0.0504,0.2016)*0.4=0.0806, \psi_3(2)=2\\
\alpha_3(3)&=&max(\alpha_2(1)\alpha_{13},\alpha_2(2)\alpha_{23},\alpha_2(3)\alpha_{33})*b_3{A}=max(0.084*0,0.252*0.2,0*1)*0.8=0.252*0.2*0.8=0.0403, \psi_3(3)=2
\end{eqnarray}$$
t=4,状态为B：
$$\begin{eqnarray}
\alpha_4(1)&=&max(\alpha_3(1)\alpha_{11},\alpha_3(2)\alpha_{21},\alpha_3(3)\alpha_{31})*b_1{B}=max(0.0235*0.4,0.0806*0,0.0403*0)*0.3=0.0235*0.4*0.3=0.0028, \psi_4(1)=1\\
\alpha_4(2)&=&max(\alpha_3(1)\alpha_{12},\alpha_3(2)\alpha_{22},\alpha_3(3)\alpha_{32})*b_2{B}=max(0.0235*0.6,0.0806*0.8,0.0403*0)*0.4=max(0.0141,0.0645)*0.6=0.0645*0.6=0.0387, \psi_4(2)=2\\
\alpha_4(3)&=&max(\alpha_3(1)\alpha_{13},\alpha_3(2)\alpha_{23},\alpha_3(3)\alpha_{33})*b_3{B}=max(0.0235*0,0.0806*0.2,0.0403*1)*0.2=max(0.0161+0.0403)*0.2=0.0403*0.2=0.0081, \psi_4(3)=3\\
&\therefore& p^*=max[0.0028, 0.0387,00.0081]=0.0387, q_4^*=2
\end{eqnarray}$$
因为第二个数是最大数
$$\begin{eqnarray}
q_3^*&=&\psi_4(q_4^*)=\psi_4(2)=2\\
q_2^*&=&\psi_3(q_3^*)=\psi_3(2)=2\\
q_1^*&=&\psi_2(q_2^*)=\psi_2(2)=1
\end{eqnarray}$$
$\therefore 1-2-2-2$是最优路径

In [ ]:
import numpy as np

### 维比特算法

# 红 白 红
A = np.array([[0.5,0.2,0.3],
              [0.3,0.5,0.2],
              [0.2,0.3,0.5]])

B = np.array([[0.5,0.5],
              [0.4,0.6],
              [0.7,0.3]])

pi = np.array([[0.2,0.4,0.4]]).T


theta_1 = [pi[i]*B[i,0] for i in range(3)]
theta_1_index = np.argmax([pi[i]*B[i,0] for i in range(3)])


print(theta_1_index)

theta_2 = [max([theta_1[j]*A[j,i]*B[i,1] for j in range(3)]) for i in range(3)]
theta_2_index = [np.argmax([theta_1[j]*A[j,i]*B[i,1] for j in range(3)]) for i in range(3)]


theta_3 = [max([theta_2[j]*A[j,i]*B[i,0] for j in range(3)]) for i in range(3)]
theta_3_index = [np.argmax([theta_2[j]*A[j,i]*B[i,0] for j in range(3)]) for i in range(3)]


theta_4 = [max([theta_3[j]*A[j,i]*B[i,1] for j in range(3)]) for i in range(3)]
theta_4_index = [np.argmax([theta_3[j]*A[j,i]*B[i,1] for j in range(3)]) for i in range(3)]


## 那么最优路径的解为

p4 = np.argmax(theta_4_index)
p3 = theta_4_index[p4]
p2 = theta_3_index[p3]
p1 = theta_2_index[p2]
print(p1,p2,p3,p4)

2
2 2 2 2


##4.3 学习问题-EM与Baum Welch方法